In [124]:
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
import numpy as np
import optuna

In [102]:
df_train = pd.read_csv("../data_preprocess/train_preprocess.csv", sep=",", header=0, na_values=['?', '', 'NA'])
# Cargamos csv con los datos de test
df_test = pd.read_csv("../data_preprocess/test_preprocess.csv", sep=",", header=0, na_values=['?', '', 'NA'])

In [103]:
df_train.iloc[:,1:36]

,X1,X2,X3,X5,X6,X7,X8,X9,X10,X12,...,X29,X30,X32,X33,X34,X35,X36,X37,X38,X39
0,-0.283105,-0.295019,-0.298533,-0.216731,-0.246022,-0.281493,-0.207331,1.398353,-0.246103,-0.172812,...,-0.451738,0,-0.128418,-0.189660,-0.372249,-0.123599,-0.398377,-0.156239,2.864004,-1.493411
1,0.544414,0.485299,0.330613,0.849708,0.854588,0.479761,0.062955,0.980651,1.765951,2.742616,...,-0.060458,0,1.172363,1.089853,0.577876,0.547511,0.172548,0.538236,0.447551,0.286241
2,-0.109076,-0.130923,-0.014559,-0.165846,0.187234,-0.000198,-0.252942,-0.310319,-0.100802,0.015777,...,1.289902,0,0.312988,0.412497,0.023685,0.606281,-0.260056,0.566045,-0.246029,-0.581444
3,-0.142328,-0.256744,-0.252773,-0.160334,-0.100531,-0.089851,-0.259658,-0.009279,0.034459,0.447257,...,-0.434911,0,0.380371,0.350567,-0.150597,-0.084419,0.354622,0.244054,1.829387,-0.269900
4,-0.248911,-0.246788,-0.146895,-0.314106,-0.185222,-0.194442,-0.339221,-0.252107,-0.150591,0.094295,...,0.161421,0,0.637207,0.754216,0.081308,0.565182,0.422371,1.122942,-0.551973,-0.319979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,0.538190,1.599296,0.912525,0.188801,0.266323,0.632552,-0.147811,-0.171094,-0.361831,-3.029536,...,2.021516,0,-0.395996,-0.439038,-0.921962,-1.023932,-0.772406,-0.412367,-0.556728,-0.077491
902,-0.310287,-0.294003,-0.287962,-0.357120,-0.306520,-0.321784,-0.233085,-0.551324,-0.427581,-0.398092,...,-0.454465,0,-1.835449,-1.413326,-0.948475,-0.761183,-0.549400,-3.399561,1.338374,-0.798629
903,3.470347,5.095952,3.636751,2.029403,-0.036367,7.164578,1.340059,0.278583,1.319153,-0.991745,...,0.310271,-1,-1.253418,-3.037877,-0.336309,-0.464607,-0.245236,-1.622027,0.029291,-0.187665
904,-0.029073,0.113556,0.108613,-0.283810,-0.538635,-0.229721,0.892070,-0.476467,-1.364114,-0.861860,...,0.041037,0,-2.270996,-1.610174,-2.246222,-1.987983,0.368031,-6.182583,0.017309,-1.425936


In [104]:
df_train.iloc[:,20:28]

,X22,X23,X24,X25,X27,X28,X29,X30
0,0.483718,-0.497176,0.0,0,-1.763188,-0.120785,-0.451738,0
1,0.249714,-0.017647,1.0,1,0.503571,0.384563,-0.060458,0
2,-0.308624,-0.447294,-1.0,0,-0.323889,1.031280,1.289902,0
3,-0.034364,0.247294,2.0,0,-0.652845,0.023290,-0.434911,0
4,-0.167567,-0.988000,1.0,0,-0.386547,0.145701,0.161421,0
...,...,...,...,...,...,...,...,...
901,0.005891,-0.344235,2.0,0,-0.162635,-0.295193,2.021516,0
902,0.875470,-0.948941,0.0,0,-1.068417,-0.687339,-0.454465,0
903,-1.242677,0.918824,-2.0,0,-0.041926,-0.908869,0.310271,-1
904,1.023400,0.166353,1.0,0,-1.537434,-1.634123,0.041037,0


In [63]:
sm=SMOTENC(categorical_features=[27,23])
x_over, y_over=sm.fit_resample(df_train.iloc[:,1:36],df_train.iloc[:,36])

In [ ]:
sm=TomekLinks()
x_over,y_over=sm.fit_resample(df_train.iloc[:,1:36],df_train.iloc[:,36])

In [105]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(df_train.iloc[:,1:36],df_train.iloc[:,36],shuffle=True)

In [109]:
unique,counts = np.unique(y_train,return_counts=True)
counts

array([ 56, 284, 175, 164])

In [110]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(x_over,y_over,shuffle=True)

In [79]:
sm=SMOTENC(categorical_features=[27,23])
x_over, y_over=sm.fit_resample(x_train,y_train)

In [111]:
sm=TomekLinks()
x_over,y_over=sm.fit_resample(x_train,y_train)

In [113]:
sm=SMOTETomek()
x_over,y_over=sm.fit_resample(x_train,y_train)

In [114]:
unique,counts = np.unique(y_over,return_counts=True)
counts

array([180, 178, 179, 181])

## Optuna para optimización de parametros

In [139]:
def objective(trial):
    solver=trial.suggest_categorical("solver",['lbfgs','newton-cg','sag','saga'])
    if solver!='liblinear':
        multiclass=trial.suggest_categorical("multi_class",['ovr','multinomial'])
    else:
        multiclass='ovr'
    multiclass_weights=trial.suggest_categorical("class_weight",['balanced',"None"])
    if multiclass_weights=="None":
        multiclass_weights=None
    fit_intercept=trial.suggest_categorical("fit_intercept",[True,False])
    if solver=="lbfgs" or solver=="newton-cg" or solver=="sag":
        penalty=trial.suggest_categorical("penalty",['l2',None])
    if solver=="saga":
        penalty="elasticnet"
        l1_ratio=trial.suggest_float("l1_ratio",0.01,0.99,step=0.01)
    else:
        l1_ratio=None
    if penalty!=None:
        C_reg=trial.suggest_float("C",0.01,2,step=None,log=True)
    else:
        C_reg=1.0
        


    lr=LogisticRegression(penalty=penalty,solver=solver,multi_class=multiclass,class_weight=multiclass_weights,fit_intercept=fit_intercept,C=C_reg,l1_ratio=l1_ratio,max_iter=10000)
    score=cross_val_score(lr,x_over,y_over,cv=5)
    accuracy=score.mean()
    return accuracy




In [140]:
study=optuna.create_study(direction="maximize")
study.optimize(objective,n_trials=1000)

[I 2024-01-09 18:55:53,830] A new study created in memory with name: no-name-ef3476b1-79c8-4e7e-8854-6447005b0dca
[I 2024-01-09 18:56:04,849] Trial 0 finished with value: 0.6282051282051283 and parameters: {'solver': 'saga', 'multi_class': 'multinomial', 'class_weight': 'None', 'fit_intercept': True, 'l1_ratio': 0.87, 'C': 0.027519724539279016}. Best is trial 0 with value: 0.6282051282051283.
[I 2024-01-09 18:56:07,798] Trial 1 finished with value: 0.7382770007770009 and parameters: {'solver': 'newton-cg', 'multi_class': 'multinomial', 'class_weight': 'balanced', 'fit_intercept': False, 'penalty': None}. Best is trial 1 with value: 0.7382770007770009.
[I 2024-01-09 18:56:11,518] Trial 2 finished with value: 0.7564296814296814 and parameters: {'solver': 'newton-cg', 'multi_class': 'multinomial', 'class_weight': 'None', 'fit_intercept': True, 'penalty': None}. Best is trial 2 with value: 0.7564296814296814.
[I 2024-01-09 18:56:20,447] Trial 3 finished with value: 0.7550213675213675 and p

In [141]:
study.best_params

{'solver': 'lbfgs',
 'multi_class': 'multinomial',
 'class_weight': 'None',
 'fit_intercept': True,
 'penalty': 'l2',
 'C': 0.9849379359164123}

In [144]:
lr=LogisticRegression(penalty="l2",solver='lbfgs',multi_class='multinomial',class_weight=None,fit_intercept=True,C=0.9849379359164123,max_iter=10000)
lr.fit(x_over,y_over)

LogisticRegression(C=0.9849379359164123, max_iter=10000,
                   multi_class='multinomial')

In [146]:
y_pred=lr.predict(x_val)
accuracy_score(y_pred,y_val)


0.6712328767123288

In [147]:
pred=lr.predict(df_test.iloc[:,1:36])
results={'ID':df_test.iloc[:,0], 'RATE': pred}
df_submission=pd.DataFrame(data=results)
df_submission.to_csv("SubmissionOPTUNANONPREPROCESS.csv",index=False)

In [148]:
unique,counts = np.unique(y_train,return_counts=True)
counts

array([15, 60, 37, 34])

In [121]:
final_lr=LogisticRegression(max_iter=100000,class_weight=None,solver='lbfgs',multi_class="multinomial",penalty='l2')
final_lr.fit(x_over,y_over)
pred=final_lr.predict(df_test.iloc[:,1:36])
results={'ID':df_test.iloc[:,0], 'RATE': pred}
df_submission=pd.DataFrame(data=results)
df_submission.to_csv("SubmissionSMOTETLink2.csv",index=False)



In [122]:
unique,counts = np.unique(pred,return_counts=True)
counts

array([ 46, 133, 118,  92])